In [1]:
import pysplit
import pandas as pd
import numpy as np
from wrf import getvar, get_basemap, latlon_coords
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import seaborn as sns
import argparse
import matplotlib as mpl
import xarray as xr
from itertools import product
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import shapely.geometry as sgeom
from shapely.ops import unary_union
from shapely.prepared import prep
from mpl_toolkits.basemap import Basemap
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd
import shapely
from descartes import PolygonPatch
from shapely.geometry import Point

In [2]:
can = gpd.GeoDataFrame.from_file("/glade/work/molina/DATA/ocean_shapefiles/ne_50m_geography_marine_polys.shp")

#goa
goa1 = can[can['name']== 'Gulf of Alaska']
goa2 = can[can['name']== 'North Pacific Ocean']
goa3 = can[can['name']== 'Cook Inlet']
#bs
bs1 = can[can['name']== 'Bering Sea']
bs2 = can[can['name']== 'Bristol Bay']
#ao
ao1 = can[can['name']== 'Arctic Ocean']
ao2 = can[can['name']== 'Beaufort Sea']
ao3 = can[can['name']== 'Chukchi Sea']
ao4 = can[can['name']== 'Amundsen Gulf']

In [3]:
land_shp_fname = shpreader.natural_earth(resolution='50m', category='physical', name='land')
land_geom = unary_union(list(shpreader.Reader(land_shp_fname).geometries()))
land = prep(land_geom)
def is_land(x, y):
    return land.contains(sgeom.Point(x, y))

In [4]:
wrffile = Dataset('/glade/scratch/molina/basile/wrfout_d01_2002-12-30_18:00:00.nc4')
tfile = getvar(wrffile, 'tk')
m = get_basemap(tfile)
lats, lons = latlon_coords(tfile)

In [33]:
which_climate='pgw'
which_month='may'
which_region='subregion6'

In [34]:
#trajgroup = pysplit.make_trajectorygroup(
#    f'/glade/scratch/molina/basile/{which_climate}_traj_{which_month}/trajid*_{which_region}_ens*_{which_month}*summer*')
#print(trajgroup.trajcount)
trajgroup = pysplit.make_trajectorygroup(
    f'/glade/scratch/molina/basile/{which_climate}_traj_{which_month}/trajid*_{which_region}_ens*_{which_month}*spring*')
print(trajgroup.trajcount)

8848


In [35]:
x_trajs = np.array([np.array(traj.path.coords[:])[-1,0] for traj in trajgroup]).flatten()
y_trajs = np.array([np.array(traj.path.coords[:])[-1,1] for traj in trajgroup]).flatten()

In [36]:
none_pointsx=[]
none_pointsy=[]
land_pointsx=[]
land_pointsy=[]
goa_pointsx=[]
goa_pointsy=[]
bs_pointsx=[]
bs_pointsy=[]
ao_pointsx=[]
ao_pointsy=[]

for x, y in zip(x_trajs, y_trajs):
    
    if is_land(x, y):
        land_pointsx.append(x)
        land_pointsy.append(y)

    elif goa1.geometry.contains(Point(x, y)).values[0]:
        goa_pointsx.append(x)
        goa_pointsy.append(y)
        
    elif goa2.geometry.contains(Point(x, y)).values[0]:
        goa_pointsx.append(x)
        goa_pointsy.append(y)
        
    elif goa3.geometry.contains(Point(x, y)).values[0]:
        goa_pointsx.append(x)
        goa_pointsy.append(y)
        
    elif bs1.geometry.contains(Point(x, y)).values[0]:
        bs_pointsx.append(x)
        bs_pointsy.append(y)
        
    elif bs2.geometry.contains(Point(x, y)).values[0]:
        bs_pointsx.append(x)
        bs_pointsy.append(y)
        
    elif ao1.geometry.contains(Point(x, y)).values[0]:
        ao_pointsx.append(x)
        ao_pointsy.append(y)
        
    elif ao2.geometry.contains(Point(x, y)).values[0]:
        ao_pointsx.append(x)
        ao_pointsy.append(y)
        
    elif ao3.geometry.contains(Point(x, y)).values[0]:
        ao_pointsx.append(x)
        ao_pointsy.append(y)
        
    elif ao4.geometry.contains(Point(x, y)).values[0]:
        ao_pointsx.append(x)
        ao_pointsy.append(y)
        
    else:
        none_pointsx.append(x)
        none_pointsy.append(y)
        

In [37]:
ds = xr.Dataset({'land_pointsx':(['a'], np.array(land_pointsx)),
                 'land_pointsy':(['a'], np.array(land_pointsy)),
                 'goa_pointsx':(['b'], np.array(goa_pointsx)),
                 'goa_pointsy':(['b'], np.array(goa_pointsy)),
                 'bs_pointsx':(['c'], np.array(bs_pointsx)),
                 'bs_pointsy':(['c'], np.array(bs_pointsy)),
                 'ao_pointsx':(['d'], np.array(ao_pointsx)),          
                 'ao_pointsy':(['d'], np.array(ao_pointsy)),
                 'none_pointsx':(['e'], np.array(none_pointsx)),
                 'none_pointsy':(['e'], np.array(none_pointsy)),
                },
                 attrs={'Trajectories and File Author':'Maria J. Molina',
                        'Trajectory Count':str(trajgroup.trajcount)})

In [38]:
ds

<xarray.Dataset>
Dimensions:       (a: 606, b: 7128, c: 60, d: 850, e: 204)
Dimensions without coordinates: a, b, c, d, e
Data variables:
    land_pointsx  (a) float64 172.7 175.7 176.6 172.9 ... -128.4 -122.4 -166.2
    land_pointsy  (a) float64 69.77 68.2 68.04 69.88 ... 65.77 63.75 63.89 68.41
    goa_pointsx   (b) float64 -161.5 -139.9 -144.0 ... -137.4 -141.4 -136.8
    goa_pointsy   (b) float64 52.88 53.15 59.76 52.83 ... 52.52 52.98 52.64
    bs_pointsx    (c) float64 -171.3 -173.9 -172.9 ... -172.6 -167.7 -168.1
    bs_pointsy    (c) float64 59.01 60.35 62.17 63.26 ... 56.84 62.2 63.24 65.46
    ao_pointsx    (d) float64 -177.6 -159.6 -177.9 ... -136.0 -136.0 -136.0
    ao_pointsy    (d) float64 73.09 74.81 69.46 74.76 ... 74.6 74.6 74.6 74.6
    none_pointsx  (e) float64 172.0 171.7 171.1 172.0 ... -131.0 -130.7 -130.8
    none_pointsy  (e) float64 70.17 70.49 70.48 70.47 ... 53.36 53.85 53.91
Attributes:
    Trajectories and File Author:  Maria J. Molina
    Trajectory Count:              8848

In [39]:
ds.to_netcdf(f'/glade/scratch/molina/basile/traj_files/originfile_{which_month}_{which_region}_{which_climate}.nc')